In [11]:
import scipy.stats
import matplotlib.pyplot as plt
import numpy as np
import functools
from tqdm import tqdm

In [26]:
# Question 5
binom = scipy.stats.binom
poisson = scipy.stats.poisson


def p(r, d, n, x, y):
    if x == n:
        return 1.0 if y == n else 0.0
    else:
        return functools.reduce(
            lambda a, k: a + poisson.pmf(mu=r * x / d, k=k + y - x) * binom.pmf(n=x, p=1 / d, k=k),
            range(max(0, x - y), x + 1),
            0.0
        )


n = 200
r = 1.1
d = 14
x0 = 50
pm = [[0 for _ in range(n + 1)] for _ in range(n + 1)]
for i in tqdm(range(n + 1)):
    i = n - i
    for j in range(n):
        pm[i][j] = p(r, d, n, i, j)
    pm[i][n] = 1 - np.sum(pm[i])  # probabilities to last state

pm = np.array(pm)
pm[0, :] = 0
pm[n, :] = 0
b = np.zeros(n + 1)
b[0] = 1
a = np.linalg.solve(np.eye(n + 1) - pm, b)
print(np.sum(a) / len(a))
print(a[50])

100%|██████████| 201/201 [05:01<00:00,  1.50s/it]

0.05277426592488677
0.007077742477949507


In [30]:
def hmm():
    MAX_STATE = 9
    x = np.random.randint(low=0, high=MAX_STATE+1) # initial location X 0
    while True:
        e = np.random.choice([-1,0,1])
        y = min(MAX_STATE, max(0, x + e)) # noisy reading of location
        yield y
        d = np.random.choice([-1,0,1], p=[1/4,1/2,1/4])
        x = min(MAX_STATE, max(0, x + d)) # new location at next timestep

ns = 10  # num states
t = [[0 for _ in range(ns)] for _ in range(ns)]  # transition matrix
for i in range(1, 9):
    t[i][i + 1] = t[i][i - 1] = 1 / 4
    t[i][i] = 1 / 2
t[9][0] = t[0][0] = 3 / 4
t[9][8] = t[0][1] = 1 / 4

e = [[0 for _ in range(ns)] for _ in range(ns)]  # emission matrix
for i in range(1, 9):
    e[i][i - 1] = e[i][i] = e[i][i + 1] = 1 / 3
e[0][0] = e[9][9] = 2 / 3
e[0][1] = e[9][8] = 1 / 3

def calc(ys):
    m = [[0 for _ in range(ns)] for _ in range(len(ys))]  # n * ns memo table
    for i in range(ns):  # initialise base case
        m[0][i] = 1 / 10 * e[i][ys[0]]
    tot = np.sum(m[0])
    for i in range(ns):
        m[0][i] /= tot
    for i in range(1, len(ys)):  # calculate inductive case
        for j in range(ns):
            for k in range(ns):
                m[i][j] += m[i - 1][k] * t[k][j]
            m[i][j] *= e[j][ys[i]]
        tot = np.sum(m[i])
        for j in range(ns):
            m[i][j] /= tot
    return m[-1]

[nan, nan, nan, nan, nan, nan, nan, nan, nan, nan]


/tmp/ipykernel_11318/259329429.py:40: RuntimeWarning: invalid value encountered in double_scalars
  m[i][j] /= tot
